In [1]:


import torch.onnx

from transformer_net import TransformerNet

# Define your paths and options here
dataset_path = "train"
style_image_path = "test/capture.png"
save_model_dir = "test"
model_name = "model.onnx"
output_image = "test/output.png"

def export_to_onnx(pth_path, onnx_path, image_size=(1280, 720)):
    """
    Load a .pth file and export the model to ONNX format.

    Parameters:
    - pth_path: Path to the .pth file containing the model's state_dict.
    - onnx_path: Path where the ONNX model should be saved.
    - image_size: Tuple indicating the height and width of the input images. Defaults to (224, 224).
    """
    # Ensure the TransformerNet class is defined or imported correctly
    transformer = TransformerNet()

    # Load the model weights
    transformer.load_state_dict(torch.load(pth_path))
    transformer.eval().cpu()  # Set to evaluation mode and move to CPU

    # Prepare a dummy input for ONNX export. The size can be dynamic
    dummy_input = torch.randn(1, 3, *image_size)  # Assuming the model expects CxHxW images

    # Export the model to ONNX with dynamic axes for flexible input dimensions
    torch.onnx.export(transformer, dummy_input, onnx_path, 
                      export_params=True, opset_version=12,
                      do_constant_folding=True, 
                      input_names=['input'], output_names=['output'],
                      dynamic_axes={'input': {0: 'batch_size', 2: 'height', 3: 'width'}, # currently will match input and output sizes, this can be separated for compression reasons
                                    'output': {0: 'batch_size', 2: 'height', 3: 'width'}})
    # TODO: Fix this so input and output dimensions can be separated, e.g. for compression reasons. We could for example render a small source image to a large canvas to save on encoding time

    print(f"Model loaded from {pth_path} and saved to {onnx_path} in ONNX format with dynamic input resolution.")

export_to_onnx(save_model_dir+'/'+"model.pth", save_model_dir+'/'+model_name)

C:\Users\brews\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\nn\modules\instancenorm.py:74: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if input.size(feature_dim) != self.num_features:
C:\Users\brews\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\nn\functional.py:2492: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if size_prods == 1:
C:\Users\brews\AppData\Local\Packages\PythonSoftwareF

Model loaded from test/model.pth and saved to test/model.onnx in ONNX format with dynamic input resolution.
